Price history

In [31]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from PIL import Image
import requests
import datetime
import re
import time
import urllib
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

url = "https://www.larvalabs.com/cryptopunks/details/1119"

In [32]:
def load_punk_info (url):
    web_r = requests.get(url)
    websoup = BeautifulSoup(web_r.text, "html.parser")
    
    price_info = pd.DataFrame()
    for titlesoup in websoup.findAll("div",{"class":"table-responsive"}):
        for element in titlesoup.findAll("tr", {"class":re.compile(r"\bpunk-history-row\b")}):
            price_info= price_info.append([element.get_text().replace("\n", "")])
    return price_info
    
def transform_punk_info (price_info):   
    price_info = price_info[0].str.split(expand=True)
    price_info["Bid"] = np.where(price_info[0].str.contains("Bid"), 1, 0)
    price_info["Offered"] = np.where(price_info[0].str.contains("Offered"), 1, 0)
    price_info["Transfer"] = np.where(price_info[0].str.contains("Transfer"), 1, 0)
    price_info["Claimed"] = np.where(price_info[0].str.contains("Claimed"), 1, 0)
    price_info["Withdrawn"] = np.where(price_info[1].str.contains("Withdrawn"), 1, 0)
    price_info["Offer"] = np.where(price_info[0].str.contains("Offer"), 1, 0)
    
    price_info["price_in_dollars"] = price_info[1].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x))
    price_info["price_in_dollars"] = np.where(price_info.Withdrawn == 1, price_info[2].apply(lambda x: re.findall(r"\(([A-Za-z0-9_$,.<]+)\)", x)), price_info["price_in_dollars"])
    price_info = price_info.explode("price_in_dollars")
    #print(price_info.head())
    price_info["Year"] = price_info[4].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x))
    price_info = price_info.explode("Year")
    
    price_info["Year"] = np.where(price_info.Year.isnull(), price_info[3].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
    price_info = price_info.explode("Year")
    
    price_info["Year"] = np.where(price_info.Year.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"^[12][0-9]{3}$", x)), price_info["Year"])
    price_info = price_info.explode("Year")
    
    price_info["Month"] = price_info[1].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x))
    price_info = price_info.explode("Month")
    
    price_info["Month"] = np.where(price_info.Month.isnull(), price_info[2].astype(str).apply(lambda x: re.findall(r"\b[a-zA-Z]{3}\b", x)), price_info["Month"])
    price_info = price_info.explode("Month")
    price_info = price_info.iloc[:,6:]
    return price_info

In [33]:
price_info = load_punk_info(url)

In [34]:
price_info = transform_punk_info(price_info)

Iterative loop

In [35]:
path = r"C:\Users\Acer\Git\deda_punks\Crypto_punks\bids"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
punks = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            punks.append(file.name)

In [36]:
import itertools
import re
new_punks = [re.findall("(\d+)", element) for element in punks]
new_punks = list(itertools.chain(*new_punks))

In [56]:
import tqdm
import time

final_df = pd.DataFrame()
for i in tqdm.tqdm(new_punks):
    try:
        url = "https://www.larvalabs.com/cryptopunks/details/" + i
        price_info = load_punk_info(url)
        price_info = transform_punk_info(price_info)
        price_info["Punk_ID"] = i
        final_df = final_df.append(price_info)
        time.sleep(3)
    except:
        print("Fuck!", i)

  0%|▏                                                                                 | 2/909 [00:05<38:34,  2.55s/it]

Fuck! 0017


  0%|▎                                                                                 | 3/909 [00:05<24:25,  1.62s/it]

Fuck! 0039


  0%|▎                                                                                 | 4/909 [00:06<17:56,  1.19s/it]

...

Fuck! 9995


100%|███████████████████████████████████████████████████████████████████████████████▉| 908/909 [07:06<00:00,  3.50it/s]

Fuck! 9999


100%|████████████████████████████████████████████████████████████████████████████████| 909/909 [07:06<00:00,  2.13it/s]

Fuck! 16


In [57]:
final_df.Punk_ID.nunique()

10

In [59]:
final_df.to_csv(r"Crypto_Punks_with_prices.csv", index = None)